In [48]:
import torch
import torch.nn as nn 
import os
import cv2
from torch.utils.data import Dataset, DataLoader

from network.network_pro import Inpaint
from losses.perceptual import PerceptualLoss

class InpaintingDataset(Dataset):
    def __init__(self, img_dir, mask_dir):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.imgs = os.listdir(img_dir)
        self.masks = os.listdir(mask_dir)
    
    def __len__(self):
        return len(self.imgs)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.imgs[idx])
        mask_path = os.path.join(self.mask_dir, self.masks[idx])

        img = cv2.imread(img_path) / 255.0 # (256, 256, 3)
        mask = cv2.imread(mask_path)[..., 0] / 255.0 # (256, 256, 3) => (256, 256)

        img = torch.Tensor(img).permute(2, 0, 1).float()
        mask = torch.Tensor(mask).unsqueeze(0).float()

        return (img, mask), img                     # img (3, 256, 256), mask (1, 256, 256)

train_dataset = InpaintingDataset('./samples/test_img_face', './samples/test_mask_face')
train_loader = DataLoader(train_dataset, batch_size=2)

def train(epochs, model, train_loader, criterion, optimizer, device):
    model.to(device)
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, targets in train_loader:
            imgs, masks = inputs[0].to(device), inputs[1].to(device)
            targets = targets.to(device)

            optimizer.zero_grad()
            outputs = model(imgs, masks)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        epoch_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")
# configs
epochs = 10
model = Inpaint()
criterion = PerceptualLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
device = 'cpu'


train(epochs, model, train_loader, criterion, optimizer, device)




/Users/huyton/opt/anaconda3/lib/python3.9/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/Users/huyton/opt/anaconda3/lib/python3.9/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3550.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.48235, 0.45882, 0.40784]
    std=[0.00392156862745098, 0.00392156862745098, 0.00392156862745098]
    interpolation=InterpolationMode.BILINEAR
)
Epoch 1/10, Loss: 121.8297
Epoch 2/10, Loss: 68.9590
Epoch 3/10, Loss: 67.7728
Epoch 4/10, Loss: 66.5114
Epoch 5/10, Loss: 58.0027
Epoch 6/10, Loss: 53.3400
Epoch 7/10, Loss: 54.5931
Epoch 8/10, Loss: 52.0841
Epoch 9/10, Loss: 49.1581
Epoch 10/10, Loss: 49.1417


In [49]:
model.eval()
test_image = cv2.imread('./samples/test_img_face/1.png') / 255.0
test_mask = cv2.imread('./samples/test_mask_face/1.png')[..., 0] / 255.0
test_image = torch.Tensor(test_image).permute(2, 0, 1).float()
test_mask = torch.Tensor(test_mask).unsqueeze(0).float()

output = model(test_image.unsqueeze(0), test_mask.unsqueeze(0))
output = output.squeeze(0).permute(1, 2, 0).detach().numpy() * 255
cv2.imwrite('./temp/output.png', output)

False

In [53]:
cv2.imwrite('./temp/test_image.png', output)

True